In [15]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [16]:
driver = webdriver.Chrome()
url_pvehicular = "https://dgii.gov.do/estadisticas/parquevehicular/Paginas/default.aspx"
driver.get(url_pvehicular)
driver.maximize_window()

sleep(1)  # Esperar un segundo entre cada scroll

# try:
#     driver.execute_script("window.scrollBy(0, 500);")  # Desplazarse 1000 píxeles hacia abajo
# except:
#     print('No he podido descender')

try: 
    driver.find_element("xpath", '/html/body/form/div[4]/div/div/div/div[2]/span/div[1]/div/div/div[2]/div/div[2]/div/div[1]/div[1]/div/div[1]/div/div/div/div/ul/li[1]/a/div/div[2]').click()
except:
    print('No encuentro el botón para descargar')

# driver.find_element('css selector', '#search-form-input').send_keys('florencia', Keys.ENTER)

Ya tenemos la hoja de excel descargada, con los datos del parque vehicular.

In [31]:
df = pd.read_excel('C:\\Users\\DELL\\Downloads\\ParqueVehicular2023\\ParqueVehicular2023.xlsx', sheet_name='2.3.1', skiprows=11, index_col=0)

In [34]:
df

,Provincia,Tipo de Vehículo,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Total,Total %
NaN,NaN,Automóviles,Autobuses,Jeep,Carga,Motocicletas,Volteo,Máquinas Pesadas,Otros,NaN,NaN
NaN,Azua,7641,1352,4935,6559,52576,341,129,129,73662.0,1.267655
NaN,Bahoruco,2288,388,1326,1114,11407,97,22,25,16667.0,0.286824
NaN,Barahona,6637,1081,5175,3916,44324,262,78,208,61681.0,1.061473
NaN,Dajabón,2515,222,2421,2290,26624,114,129,57,34372.0,0.591510
NaN,Distrito Nacional,446535,49784,227089,208389,772371,9328,12329,11920,1737745.0,29.904982
NaN,Duarte,23338,2087,13417,12810,120644,493,1398,292,174479.0,3.002622
NaN,El Seibo,2341,396,2255,2085,21761,126,106,42,29112.0,0.500991
NaN,Elías Piña,1194,165,702,505,4866,41,33,5,7511.0,0.129257
NaN,Espaillat,20213,1789,16131,12704,78976,546,270,251,130880.0,2.252324
